In [1]:
from sqlalchemy import create_engine, exc, text

DB_TYPE = 'mysql'
DB_DRIVER = 'pymysql'
DB_USER = 'root'
DB_PASS = 'R1a2m3$%^'
DB_HOST = 'localhost'
DB_PORT = '3306'
DB_NAME = 'traveldb'

DATABASE_URL = f"{DB_TYPE}+{DB_DRIVER}://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

try:
    print(f"Attempting to connect to database at {DB_HOST}:{DB_PORT}")
    engine = create_engine(DATABASE_URL)
    # Test the connection
    with engine.connect() as connection:
        connection.execute(text("SELECT 1"))
    print("Database connection successful.")
except exc.SQLAlchemyError as e:
    print(f"Error connecting to the database: {e}")


Attempting to connect to database at localhost:3306
Database connection successful.


In [ ]:
from flask import Flask, render_template_string
import pandas as pd
from sqlalchemy import create_engine, text
from threading import Thread  # Import the Thread class

# Create the Flask application
app = Flask(__name__)

# Replace these variables with your actual database credentials
DB_TYPE = 'mysql'
DB_DRIVER = 'pymysql'
DB_USER = 'root'
DB_PASS = 'R1a2m3$%^'
DB_HOST = 'localhost'
DB_PORT = '3306'
DB_NAME = 'traveldb'

# Create the database engine
DATABASE_URL = f"{DB_TYPE}+{DB_DRIVER}://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

try:
    print(f"Attempting to connect to database at {DB_HOST}:{DB_PORT}")
    engine = create_engine(DATABASE_URL)
    # Test the connection
    with engine.connect() as connection:
        connection.execute(text("SELECT 1"))
    print("Database connection successful.")
except Exception as e:
    print(f"Error connecting to the database: {e}")
    engine = None

@app.route('/')
def index():
    if engine is None:
        return "Error connecting to the database."
    
    try:
        # Query to retrieve the total searches data
        query_searches = """
        SELECT `Searches`
        FROM `All-time Table-All Cities`;
        """
        
        # Execute the query and load the result into a DataFrame
        df_searches = pd.read_sql(query_searches, engine)
        print("Searches DataFrame:")
        print(df_searches.head())  # Log the head of the DataFrame for debugging
        
        # Remove the last cell of the 'Searches' column
        if not df_searches.empty:
            total_searches_without_last = df_searches['Searches'].iloc[:-1].sum()
        else:
            total_searches_without_last = 0

        print("Total Searches:", total_searches_without_last)

        # Query to retrieve the searches which got quotes data
        query_quotes = """
        SELECT `Searches which got Quotes`
        FROM `All-time Table-All Cities`;
        """
        
        # Execute the query and load the result into a DataFrame
        df_quotes = pd.read_sql(query_quotes, engine)
        print("Quotes DataFrame:")
        print(df_quotes.head())  # Log the head of the DataFrame for debugging
        
        # Remove the last cell of the 'Searches which got Quotes' column
        if not df_quotes.empty:
            searchGotQuote_without_last = df_quotes['Searches which got Quotes'].iloc[:-1].sum()
        else:
            searchGotQuote_without_last = 0

        print("Total Searches that Resulted in Quotes:", searchGotQuote_without_last)

        return render_template_string("""
            <!doctype html>
            <html lang="en">
              <head>
                <!-- Required meta tags -->
                <meta charset="utf-8">
                <meta name="viewport" content="width=device-width, initial-scale=1">

                <!-- Bootstrap CSS -->
                <link href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css" rel="stylesheet">

                <title>Searches and Quotes Data</title>
              </head>
              <body>
                <div class="container">
                  <h1 class="mt-5">Searches and Quotes Data</h1>
                  <p class="lead">Total Searches: {{ total_searches_without_last }}</p>
                  <p class="lead">Total Searches that Resulted in Quotes: {{ searchGotQuote_without_last }}</p>
                </div>

                <!-- Optional JavaScript -->
                <!-- jQuery first, then Popper.js, then Bootstrap JS -->
                <script src="https://code.jquery.com/jquery-3.5.1.slim.min.js"></script>
                <script src="https://cdn.jsdelivr.net/npm/@popperjs/core@2.5.4/dist/umd/popper.min.js"></script>
                <script src="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/js/bootstrap.min.js"></script>
              </body>
            </html>
        """, total_searches_without_last=total_searches_without_last, searchGotQuote_without_last=searchGotQuote_without_last)

    except Exception as e:
        print(f"Error retrieving data: {e}")
        return "Error retrieving data."

def run_app():
    app.run()

# Start the Flask app in a separate thread
if __name__ == '__main__':
    thread = Thread(target=run_app)
    thread.start()


Attempting to connect to database at localhost:3306
Database connection successful.


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [02/Jul/2024 20:34:32] "GET / HTTP/1.1" 200 -


Searches DataFrame:
    Searches
0  133678174
1   16433036
2    1909082
3    2777020
4     509102
Total Searches: 156661836
Quotes DataFrame:
   Searches which got Quotes
0                   58076082
1                    6200694
2                     834920
3                     690006
4                     267197
Total Searches that Resulted in Quotes: 66276688


127.0.0.1 - - [02/Jul/2024 20:34:35] "GET /favicon.ico HTTP/1.1" 404 -


In [5]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, Column, String, BigInteger, Float, text
from sqlalchemy.exc import SQLAlchemyError

# MySQL database configuration
DB_TYPE = 'mysql'
DB_DRIVER = 'pymysql'
DB_USER = 'root'
DB_PASS = 'R1a2m3$%^'
DB_HOST = 'localhost'
DB_PORT = '3306'
DB_NAME = 'traveldb'

# File path to your CSV (adjust as per your actual path)
csv_file = r"D:\Python_Intern_project\All-time Registration Trends-Nov 01, 2022 → Jun 03, 2024.csv"
# Database connection string
connection_str = f'{DB_TYPE}+{DB_DRIVER}://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}'

# Create SQLAlchemy engine
engine = create_engine(connection_str)

# Read CSV file into pandas DataFrame
try:
    df = pd.read_csv(csv_file)
    print("CSV file read successfully.")
except Exception as e:
    print(f"Error reading CSV file: {e}")
    exit()

# Convert the 'Time' column to string format
df['Time'] = df['Time'].astype(str)

# Function to clean data and infer data types
def clean_and_infer_type(value):
    if isinstance(value, str):
        value = value.replace('"', '').replace(',', '').replace('₹', '').strip()
        if value.isdigit():
            return int(value)
        try:
            return float(value)
        except ValueError:
            return value
    return value

# Apply cleaning and type inference to the DataFrame
for col in df.columns:
    if col != 'Time':
        df[col] = df[col].apply(clean_and_infer_type)

# Define table name
table_name = 'all-time registration trends'

# Drop the existing table if it exists
try:
    with engine.connect() as connection:
        connection.execute(text(f"DROP TABLE IF EXISTS `{table_name}`"))
    print(f"Table '{table_name}' dropped successfully.")
except SQLAlchemyError as e:
    print(f"Error dropping table: {e}")
    exit()

# Define the new table schema based on the DataFrame columns
metadata = MetaData()
columns = []
for column_name, dtype in zip(df.columns, df.dtypes):
    if column_name == 'Time':
        columns.append(Column(column_name, String(255)))
    elif dtype == 'int64':
        columns.append(Column(column_name, BigInteger))
    elif dtype == 'float64':
        columns.append(Column(column_name, Float))
    else:
        columns.append(Column(column_name, String(255)))

table = Table(table_name, metadata, *columns)

# Create the table in the database
try:
    metadata.create_all(engine)
    print(f"Table '{table_name}' created successfully.")
except SQLAlchemyError as e:
    print(f"Error creating table: {e}")
    exit()

# Insert data into the new table
try:
    df.to_sql(name=table_name, con=engine, if_exists='replace', index=False)
    print("Data replaced successfully in SQL database.")
except SQLAlchemyError as e:
    print(f"Error inserting data into table: {e}")
finally:
    engine.dispose()  # Dispose of the engine resources


CSV file read successfully.
Table 'all-time registration trends' dropped successfully.
Table 'all-time registration trends' created successfully.
Data replaced successfully in SQL database.


In [12]:
#final script .........
import pandas as pd
from sqlalchemy import create_engine

# Replace these variables with your actual database credentials
DB_TYPE = 'mysql'
DB_DRIVER = 'pymysql'
DB_USER = 'root'
DB_PASS = 'R1a2m3$%^'
DB_HOST = 'localhost'
DB_PORT = '3306'
DB_NAME = 'traveldb'

# Create the database engine
DATABASE_URL = f"{DB_TYPE}+{DB_DRIVER}://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(DATABASE_URL)

def execute_query(query):
    """Executes the given SQL query and returns the result as a DataFrame."""
    return pd.read_sql(query, engine)

def exclude_last_row(df):
    """Excludes the last row from the given DataFrame."""
    return df.iloc[:-1].copy()

def calculate_rate(numerator, denominator):
    """Calculates the rate as a percentage."""
    return (numerator / denominator) * 100 if denominator > 0 else 0

# Query 1: Total Bookings Canceled by Customers
query1 = "SELECT `Bookings`, `User Cancellation Rate` FROM `All-time Table-All Cities`;"
df1 = exclude_last_row(execute_query(query1))
df1['Bookings'] = pd.to_numeric(df1['Bookings'], errors='coerce')
df1['User Cancellation Rate'] = pd.to_numeric(df1['User Cancellation Rate'], errors='coerce')
total_bookings = df1['Bookings'].sum()
customer_not_cancelled = total_bookings - (df1['Bookings'] * df1['User Cancellation Rate'] / 100).sum()
customer_cancellations = total_bookings - customer_not_cancelled
print("Total number of bookings canceled by customers:", customer_cancellations)

# Query 2: Average Distance per Trip
query2 = "SELECT `Average Distance per Trip (km)` FROM `All-time Table-All Cities`;"
df2 = exclude_last_row(execute_query(query2))
average_distance_per_trip = df2['Average Distance per Trip (km)'].mean() if not df2.empty else 0
print("Average Distance per Trip (km):", average_distance_per_trip)

# Query 3: Average Fare per Trip
query3 = "SELECT `Average Fare per Trip` FROM `All-time Table-All Cities`;"
df3 = exclude_last_row(execute_query(query3))
average_fare_per_trip = df3['Average Fare per Trip'].mean() if not df3.empty else 0
print("Average Fare per Trip:", average_fare_per_trip)

# Query 4: Top Two Locations by Trip Count
query4 = "SELECT `City`, `Completed Trips` FROM `All-time Table-All Cities`;"
df4 = exclude_last_row(execute_query(query4))
df4['Completed Trips'] = pd.to_numeric(df4['Completed Trips'], errors='coerce')
top_two_locations = df4.groupby('City')['Completed Trips'].sum().nlargest(2)
print("Top two locations by trip count:")
print(top_two_locations)

# Existing queries and their handling
queries = {
    "Total Unique Trips excluding the last cell": "SELECT `Completed Trips` FROM `All-time Table-All Cities`;",
    "Total Searches excluding the last cell": "SELECT `Searches` FROM `All-time Table-All Cities`;",
    "Total Searches that got an estimate": "SELECT `Searches which got estimate` FROM `All-time Table-All Cities`;",
    "Total Searches for quotes": "SELECT `Searches for Quotes` FROM `All-time Table-All Cities`;",
    "Total Searches that resulted in quotes": "SELECT `Searches which got Quotes` FROM `All-time Table-All Cities`;",
    "Total Distance Traveled": "SELECT `Distance Travelled (km)` FROM `All-time Table-All Cities`;",
    "Rate of Estimates to Searches for Quotes": "SELECT `Searches which got estimate`, `Searches for Quotes` FROM `All-time Table-All Cities`;",
    "Quote Acceptance Rate": "SELECT `Searches`, `Searches which got Quotes` FROM `All-time Table-All Cities`;",
    "City with the highest total fare": "SELECT `City`, `Drivers' Earnings` FROM `All-time Table-All Cities`;",
    "City with the highest number of cancellations": "SELECT `City`, `Cancelled Bookings` FROM `All-time Table-All Cities`;",
    "City with the highest number of completed trips": "SELECT `City`, `Completed Trips` FROM `All-time Table-All Cities`;",
    "Duration with highest trip count and total fare": "SELECT `Time`, `Completed Trips`, `Drivers' Earnings` FROM `all-time trip trends-all cities`;",
    "Booking Cancellation Rate": "SELECT `City`, `Bookings`, `Cancelled Bookings` FROM `All-time Table-All Cities`;"
}

# Execute each query and print the result
for description, query in queries.items():
    df = exclude_last_row(execute_query(query))
    
    if description in ["Rate of Estimates to Searches for Quotes", "Quote Acceptance Rate"]:
        if not df.empty:
            num = df.iloc[:, 0].sum()
            denom = df.iloc[:, 1].sum()
            rate = calculate_rate(num, denom)
        else:
            rate = 0
        print(f"{description}: {rate:.2f}%")
    
    elif description == "Total Distance Traveled":
        total_distance = df.iloc[:, 0].sum() if not df.empty else 0
        print(f"{description}: {total_distance}")
    
    elif description in ["City with the highest total fare", "City with the highest number of cancellations", "City with the highest number of completed trips"]:
        if not df.empty:
            col_name = df.columns[1]
            df[col_name] = pd.to_numeric(df[col_name], errors='coerce')
            max_city = df.groupby('City')[col_name].sum().idxmax()
            max_value = df.groupby('City')[col_name].sum().max()
        else:
            max_city = None
            max_value = 0
        print(f"{description}: {max_city} with {max_value}")
    
    elif description == "Duration with highest trip count and total fare":
        if not df.empty:
            df['Completed Trips'] = pd.to_numeric(df['Completed Trips'], errors='coerce')
            df['Drivers\' Earnings'] = pd.to_numeric(df['Drivers\' Earnings'], errors='coerce')
            grouped = df.groupby('Time').agg({
                'Completed Trips': 'sum',
                'Drivers\' Earnings': 'sum'
            }).reset_index()
            duration_highest_trip = grouped.loc[grouped['Completed Trips'].idxmax()]
            duration_highest_fare = grouped.loc[grouped['Drivers\' Earnings'].idxmax()]
            print(f"Duration with highest trip count: {duration_highest_trip}")
            print(f"Duration with highest total fare: {duration_highest_fare}")
        else:
            print("No data found for durations with highest trip count and total fare.")
    
    elif description == "Booking Cancellation Rate":
        if not df.empty:
            df['Bookings'] = pd.to_numeric(df['Bookings'], errors='coerce')
            df['Cancelled Bookings'] = pd.to_numeric(df['Cancelled Bookings'], errors='coerce')
            if df['Bookings'].sum() > 0:
                cancellation_rate = (df['Cancelled Bookings'].sum() / df['Bookings'].sum()) * 100
            else:
                cancellation_rate = 0
            print(f"Booking Cancellation Rate: {cancellation_rate:.2f}%")
        else:
            print("No data found for calculating the booking cancellation rate.")
    
    else:
        result = df.iloc[:, 0].sum() if not df.empty else 0
        print(f"{description}: {result}")

# Additional Queries
# Total Completed Trips
query_completed_trips = "SELECT `Completed Trips` FROM `All-time Table-All Cities`;"
df_completed_trips = exclude_last_row(execute_query(query_completed_trips))
df_completed_trips['Completed Trips'] = pd.to_numeric(df_completed_trips['Completed Trips'], errors='coerce')
total_completed_trips = df_completed_trips['Completed Trips'].sum() if not df_completed_trips.empty else 0
print("Total Completed Trips:", total_completed_trips)

# Total Drivers' Earnings
query_drivers_earnings = "SELECT `Drivers' Earnings` FROM `All-time Table-All Cities`;"
df_drivers_earnings = exclude_last_row(execute_query(query_drivers_earnings))
df_drivers_earnings["Drivers' Earnings"] = pd.to_numeric(df_drivers_earnings["Drivers' Earnings"], errors='coerce')
total_drivers_earnings = df_drivers_earnings["Drivers' Earnings"].sum() if not df_drivers_earnings.empty else 0
print("Total Drivers' Earnings:", total_drivers_earnings)


Total number of bookings canceled by customers: 92583.90690000355
Average Distance per Trip (km): 8.19125
Average Fare per Trip: 160.5675
Top two locations by trip count:
City
Bangalore    36380329
Kolkata       2985299
Name: Completed Trips, dtype: int64
Total Unique Trips excluding the last cell: 40530339
Total Searches excluding the last cell: 156661836
Total Searches that got an estimate: 142188533
Total Searches for quotes: 117780530
Total Searches that resulted in quotes: 66276688
Total Distance Traveled: 327056162
Rate of Estimates to Searches for Quotes: 120.72%
Quote Acceptance Rate: 236.38%
City with the highest total fare: Bangalore with 5149758044
City with the highest number of cancellations: Bangalore with 20829119
City with the highest number of completed trips: Bangalore with 36380329
Duration with highest trip count: Time                 Mar 07, 2024
Completed Trips            131087
Drivers' Earnings        17705278
Name: 348, dtype: object
Duration with highest total

In [2]:
from flask import Flask, render_template_string
import pandas as pd
from sqlalchemy import create_engine, text
from threading import Thread  # Import the Thread class

# Create the Flask application
app = Flask(__name__)

# Replace these variables with your actual database credentials
DB_TYPE = 'mysql'
DB_DRIVER = 'pymysql'
DB_USER = 'root'
DB_PASS = 'R1a2m3$%^'
DB_HOST = 'localhost'
DB_PORT = '3306'
DB_NAME = 'traveldb'

# Create the database engine
DATABASE_URL = f"{DB_TYPE}+{DB_DRIVER}://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

try:
    print(f"Attempting to connect to database at {DB_HOST}:{DB_PORT}")
    engine = create_engine(DATABASE_URL)
    # Test the connection
    with engine.connect() as connection:
        connection.execute(text("SELECT 1"))
    print("Database connection successful.")
except Exception as e:
    print(f"Error connecting to the database: {e}")
    engine = None

@app.route('/')
def index():
    if engine is None:
        return "Error connecting to the database."

    try:
        # Query to retrieve the total searches data
        query_searches = """
        SELECT `Searches`
        FROM `All-time Table-All Cities`;
        """

        # Execute the query and load the result into a DataFrame
        df_searches = pd.read_sql(query_searches, engine)
        print("Searches DataFrame:")
        print(df_searches.head())  # Log the head of the DataFrame for debugging

        # Remove the last cell of the 'Searches' column
        if not df_searches.empty:
            total_searches_without_last = df_searches['Searches'].iloc[:-1].sum()
        else:
            total_searches_without_last = 0

        print("Total Searches:", total_searches_without_last)

        # Query to retrieve the searches which got quotes data
        query_quotes = """
        SELECT `Searches which got Quotes`
        FROM `All-time Table-All Cities`;
        """

        # Execute the query and load the result into a DataFrame
        df_quotes = pd.read_sql(query_quotes, engine)
        print("Quotes DataFrame:")
        print(df_quotes.head())  # Log the head of the DataFrame for debugging

        # Remove the last cell of the 'Searches which got Quotes' column
        if not df_quotes.empty:
            searchGotQuote_without_last = df_quotes['Searches which got Quotes'].iloc[:-1].sum()
        else:
            searchGotQuote_without_last = 0

        print("Total Searches that Resulted in Quotes:", searchGotQuote_without_last)

        return render_template_string("""
            <!doctype html>
            <html lang="en">
              <head>
                <!-- Required meta tags -->
                <meta charset="utf-8">
                <meta name="viewport" content="width=device-width, initial-scale=1">

                <!-- Bootstrap CSS -->
                <link href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css" rel="stylesheet">

                <!-- Custom CSS -->
                <style>
                    body {
                        background-color: #f8f9fa; /* Light gray background */
                    }
                    .container {
                        margin-top: 50px;
                    }
                    .jumbotron {
                        background-color: #343a40; /* Dark background for jumbotron */
                        color: #ffffff; /* White text */
                    }
                </style>

                <title>Searches and Quotes Data</title>
              </head>
              <body>
                <div class="container">
                  <div class="jumbotron">
                    <h1 class="display-4">Searches and Quotes Data</h1>
                    <p class="lead">Total Searches: {{ total_searches_without_last }}</p>
                    <p class="lead">Total Searches that Resulted in Quotes: {{ searchGotQuote_without_last }}</p>
                  </div>
                </div>

                <!-- Optional JavaScript -->
                <!-- jQuery first, then Popper.js, then Bootstrap JS -->
                <script src="https://code.jquery.com/jquery-3.5.1.slim.min.js"></script>
                <script src="https://cdn.jsdelivr.net/npm/@popperjs/core@2.5.4/dist/umd/popper.min.js"></script>
                <script src="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/js/bootstrap.min.js"></script>
              </body>
            </html>
        """, total_searches_without_last=total_searches_without_last, searchGotQuote_without_last=searchGotQuote_without_last)

    except Exception as e:
        print(f"Error retrieving data: {e}")
        return "Error retrieving data."

def run_app():
    app.run()

# Start the Flask app in a separate thread
if __name__ == '__main__':
    thread = Thread(target=run_app)
    thread.start()


Attempting to connect to database at localhost:3306
Database connection successful.


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [07/Jul/2024 11:33:54] "GET / HTTP/1.1" 200 -


Searches DataFrame:
    Searches
0  133678174
1   16433036
2    1909082
3    2777020
4     509102
Total Searches: 156661836
Quotes DataFrame:
   Searches which got Quotes
0                   58076082
1                    6200694
2                     834920
3                     690006
4                     267197
Total Searches that Resulted in Quotes: 66276688


127.0.0.1 - - [07/Jul/2024 11:33:57] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [07/Jul/2024 11:37:05] "GET / HTTP/1.1" 200 -


Searches DataFrame:
    Searches
0  133678174
1   16433036
2    1909082
3    2777020
4     509102
Total Searches: 156661836
Quotes DataFrame:
   Searches which got Quotes
0                   58076082
1                    6200694
2                     834920
3                     690006
4                     267197
Total Searches that Resulted in Quotes: 66276688
